In [ ]:
from test_and_recommendation import user_vector, recommendation

In [10]:
from text_processing import get_text_map
from os import listdir
from os.path import join
import random
import json
from collections import OrderedDict


In [20]:
import math

ПРЕДОБРАБАТЫВАЕМ ТЕКСТЫ ПОДГОТОВЛЕННЫМ АЛГОРИТМОМ get_text_map

texts = []
for file in listdir("./for_test"):
    if file.endswith(".txt"):
        print(file)
        line = ''
        with open(join("./for_test", file), 'r', encoding = "utf-8") as f:
            for l in f.readlines():
                line += l + ' '
            line = line.strip()
        #print(line)
        text_map = get_text_map(join("./for_test", file))
        #text_map = get_text_map(line,raw_text_input = True)
        with open(join("./for_test", file) + ".json", "w") as f:
            json.dump(text_map,f, indent = 4, ensure_ascii = False) 
        texts.append(text_map)

In [3]:
text_maps_json = []
for file in listdir("./for_test"):
    if file.endswith(".json"):
        with open (join("./for_test", file), "r", encoding='utf-8') as f:
            text_map = json.load(f)
            text_maps_json.append(text_map)

In [39]:
class user_vector:
    def __init__(self,debug = False):
        self.debug = debug
        self.vocab_features =[]
        self.sentence_features =[]
        #coreference_items, negation_items, sent_special_pos, dependencies_length, Y (answer)                   
        self.text_fearues = [] #OrderedDict([("lix",[]),("ttr",[])])
        self.answers_count = OrderedDict([("correct_answers",0),("incorrect_answers",0)])
        self.trigramms_list = []
    def start_new_text(self):
        self.answers_count['correct_answers'] = 0
        self.answers_count['incorrect_answers'] = 0
        if self.debug:print("answers count has been reset", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
    
    def end_text(self, text_map):
        if self.debug:
            print("\n========")
            print("SUM UP TEXT VALUES")
            print("========\n")
        correct_answers_rate = round(self.answers_count['correct_answers'] / (self.answers_count['correct_answers'] 
                                                                              + self.answers_count['incorrect_answers']),2)
        current_text_features = []
        if self.debug: 
            print("answers_count", self.answers_count)
        current_text_features.append(text_map['lix'])
        current_text_features.append(text_map['ttr'])
        #current_text_features.extend(text_map['vocab_properties'])
        current_text_features.extend(text_map['sent_properties'])
        current_text_features.append(correct_answers_rate)
        self.text_fearues.append(current_text_features)
        if self.debug: print("TEXT FEATURES",self.text_fearues)
    
    def update_vector_with_answer_sentence(self, sentence_map, effected_collocations_start_indexes_list, correct_answer):
        #update setnence and text features
        if self.debug:
            print("\n===NEW REPLY CALCULATION====")
            print("\n========")
            print("ADDING SENTENCE RESULTS")
            print("========\n")
            
        #update setnence and text features
        if correct_answer == True:
            answer_value = 1
            self.answers_count['correct_answers'] += 1
            if self.debug: print("Answer for this question is correct")
        else:
            answer_value = 0
            self.answers_count['incorrect_answers'] += 1
            if self.debug: print("Answer for this question is incorrect")
        if self.debug:print("check answers count", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
        current_sentence_features = []
        current_sentence_features.append(sentence_map['spec_sentence_features']['negation'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['coreference'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['vozvr_verb'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['prich'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['deepr'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['case_complexity'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['mean_depend_length'])
        #current_sentence_features.extend(sentence_map['average_vocabulary'])
        current_sentence_features.append(answer_value)#target variable
        self.sentence_features.append(current_sentence_features)
        
        if self.debug: print("SENTENCE FEATURES", current_sentence_features)
        
        if self.debug:
            print("\n========")
            print("ADDING VOCABULARY RESULTS")
            print("========\n") 
            
        for word_w in sentence_map['collocation_vectors_list']:
            if word_w[0] in effected_collocations_start_indexes_list:
                current_word_features = []
                print(word_w[0],word_w[1][0])
                current_word_features.append(str("local_freq_") + str(word_w[1][1]))
                current_word_features.append(str("global_freq_mpi_ln_") + str(math.log(word_w[1][2])))
                current_word_features.extend(word_w[2][0])
                current_word_features.append(answer_value)
                #print(current_word_features)
                
                
DEBUG = True
user = user_vector(debug = DEBUG)
user.start_new_text()
user.update_vector_with_answer_sentence(text_maps_json[0]['sentences_map'][0], 
                                        effected_collocations_start_indexes_list = [0,3,6],
                                        correct_answer = True)
user.update_vector_with_answer_sentence(text_maps_json[0]['sentences_map'][1], 
                                        effected_collocations_start_indexes_list = [0,1,3,6,7,8],
                                        correct_answer = False)


answers count has been reset 0 0

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer for this question is correct
check answers count 1 0
SENTENCE FEATURES [0, 0, 0, 0, 0, 0, 0.25333333333333335, 1]

ADDING VOCABULARY RESULTS

0 в один из
3 ночной клуб хмельницкий
6 группа профессиональный боксер

===NEW REPLY CALCULATION====

ADDING SENTENCE RESULTS

Answer for this question is incorrect
check answers count 1 1
SENTENCE FEATURES [0, 0, 0, 0, 0.04262100983035508, 0.09396631143511165, 0.29545454545454547, 0]

ADDING VOCABULARY RESULTS

0 как
1 рассказывать в
3 городской милиция повод
6 для
7 драка
8 становиться


In [ ]:
DEBUG = False
user = user_vector(debug = DEBUG)
for text in texts:
    #print("NEW TEXT")
    user.start_new_text()
    answers = []  
    
    #print(len(text['sentences_map']))
    mark_sentences_indexes = random.sample(range(len(text['sentences_map'])), min(len(text['sentences_map']),5))
    #print("mark_sentences_indexes", mark_sentences_indexes)
    answers = [0,1,1,1,0]
    """
    positive_count = random.randint(0, 4)
    for i in range(positive_count):
        answers[i] = 1
    #for j in range(len(mark_sentences_indexes)): 
        #answers.append(random.randint(0, 1))"""
    
    for sentence_ind,answer in zip(mark_sentences_indexes,answers):
        #print(sentence_ind)
        user.update_vector_with_answer_sentence(text['sentences_map'][sentence_ind], correct_answer = answer)
    user.end_text(text)
vocab_model, sentence_model, text_model = user.export_user_vector()  

In [ ]:
class user_vector:
    def __init__(self,debug = False):
        self.debug = debug
        self.vocab_features =[]
        self.sentence_features =[]
        #coreference_items, negation_items, sent_special_pos, dependencies_length, Y (answer)                   
        self.text_fearues = [] #OrderedDict([("lix",[]),("ttr",[])])
        self.answers_count = OrderedDict([("correct_answers",0),("incorrect_answers",0)])
        self.trigramms_list = []
        
    def start_new_text(self):
        self.answers_count['correct_answers'] = 0
        self.answers_count['incorrect_answers'] = 0
        if self.debug:print("answers count has been reset", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
        
    def end_text(self, text_map):
        if self.debug:
            print("\n========")
            print("SUM UP TEXT VALUES")
            print("========\n")
        correct_answers_rate = round(self.answers_count['correct_answers'] / (self.answers_count['correct_answers'] 
                                                                              + self.answers_count['incorrect_answers']),2)
        current_text_features = []
        if self.debug: 
            print("answers_count", self.answers_count)
        current_text_features.append(text_map['lix'])
        current_text_features.append(text_map['ttr'])
        #current_text_features.extend(text_map['vocab_properties'])
        current_text_features.extend(text_map['sent_properties'])
        current_text_features.append(correct_answers_rate)
        self.text_fearues.append(current_text_features)
        if self.debug: print("TEXT FEATURES",self.text_fearues)
        
    def update_vector_with_answer_sentence(self, sentence_map, correct_answer):
        #update setnence and text features
        if self.debug:
            print("\n===NEW REPLY CALCULATION====")
            print("\n========")
            print("ADDING SENTENCE RESULTS")
            print("========\n")
            
        #update setnence and text features
        if correct_answer == True:
            answer_value = 1
            self.answers_count['correct_answers'] += 1
            if self.debug: print("Answer for this question is correct")
        else:
            answer_value = 0
            self.answers_count['incorrect_answers'] += 1
            if self.debug: print("Answer for this question is incorrect")
        if self.debug:print("check answers count", self.answers_count['correct_answers'], self.answers_count['incorrect_answers'])
        current_sentence_features = []
        current_sentence_features.append(sentence_map['spec_sentence_features']['negation'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['coreference'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['vozvr_verb'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['prich'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['deepr'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['case_complexity'])
        current_sentence_features.append(sentence_map['spec_sentence_features']['mean_depend_length'])
        #current_sentence_features.extend(sentence_map['average_vocabulary'])
        current_sentence_features.append(answer_value)#target variable
        self.sentence_features.append(current_sentence_features)
        
        if self.debug: print("SENTENCE FEATURES", current_sentence_features)
        
        if self.debug:
            print("\n========")
            print("ADDING VOCABULARY RESULTS")
            print("========\n")
            
        understanding_importance_list = []
        understanding_importance_sum = 0
        for word_w in sentence_map['sentence_words']:
            understanding_importance = word_w['vocabulary_prop']['tf_idf']
            understanding_importance_sum += understanding_importance
            understanding_importance_list.append([word_w['lemma'], understanding_importance,word_w['lex_vector'],word_w['lex_trigram']])
            
        for un_unit in understanding_importance_list:
            if(understanding_importance_sum > 0):
                un_unit[1] /= understanding_importance_sum
        #if self.debug:print("understanding_importance_list", understanding_importance_list)
                       
        
        for unit_index in range(len(understanding_importance_list) ):
            current_element = understanding_importance_list[unit_index][2]
            """
            left_unit_index = unit_index - 1
            if left_unit_index <0:
                left_element = 300 * [0] 
            else:
                left_element = understanding_importance_list[left_unit_index][2]
            right_unit_index = unit_index + 1
            if right_unit_index >=  len(understanding_importance_list):
                right_element = 300 * [1]
            else:
                right_element = understanding_importance_list[right_unit_index][2]
             """   
            current_lex_vector = []
            #print("current_element", current_element)
            current_lex_vector.extend(current_element)
            
            if (correct_answer): 
                current_lex_vector.append(understanding_importance_list[unit_index][1])
                self.trigramms_list.append(understanding_importance_list[unit_index][3])
                #current_lex_vector.append(understanding_importance_list[unit_index][3])
                self.vocab_features.append(current_lex_vector)
                #print("current_lex_vector", current_lex_vector)
            else:
                current_lex_vector.append(-1 * understanding_importance_list[unit_index][1])
                self.trigramms_list.append(understanding_importance_list[unit_index][3])
                #current_lex_vector.append(understanding_importance_list[unit_index][3])
                self.vocab_features.append(current_lex_vector)

# СИМУЛИРУЕМ ПРОЦЕСС ПРОХОЖДЕНИЯ ТЕСТА НА ПРЕДОБРАБОТАННЫХ ТЕКСТАХ

In [ ]:
DEBUG = False
user = user_vector(debug = DEBUG)
for text in texts:
    #print("NEW TEXT")
    user.start_new_text()
    answers = []  
    
    #print(len(text['sentences_map']))
    mark_sentences_indexes = random.sample(range(len(text['sentences_map'])), min(len(text['sentences_map']),5))
    #print("mark_sentences_indexes", mark_sentences_indexes)
    answers = [0,1,1,1,0]
    """
    positive_count = random.randint(0, 4)
    for i in range(positive_count):
        answers[i] = 1
    #for j in range(len(mark_sentences_indexes)): 
        #answers.append(random.randint(0, 1))"""
    
    for sentence_ind,answer in zip(mark_sentences_indexes,answers):
        #print(sentence_ind)
        user.update_vector_with_answer_sentence(text['sentences_map'][sentence_ind], correct_answer = answer)
    user.end_text(text)
vocab_model, sentence_model, text_model = user.export_user_vector()  

СТРОИМ МОДЕЛЬ НОВОГО ТЕКСТА ПОД РЕКОМЕНДАЦИЮ

In [ ]:
with open("text_8.txt", 'r', encoding = "utf-8") as f:
    for l in f.readlines():
        line += l + ' '
    line = line.strip()
    text_map = get_text_map(line)

In [ ]:
recommendation(texts[8],vocab_model,sentence_model,text_model, debug = False)

In [ ]:
(%понимаемых слов - 0.85, показатель уверенности в предсказании)
средняя вероятность понимания предложения - 0.85
%вопросом на которые человек сможет ответить - 0.85